# Machine Problem 3: NumPy CNN

### Import Libraries

In [1]:
import numpy as np
import os
import time
from scipy import signal
from imageio import imread
from random import shuffle
from matplotlib import pyplot as plt

%matplotlib inline

### Preprocessing Functions

In [2]:
# load_images
    # Read in images and makes a list for each set in the form: [images, labels]
    # images: np array with dims [N x img_height x img width x num_channels]
    # labels: np array with dims [N x 1]. elephant = 0, lionfish = 1
    #
    # Returns:  train_set: The list [train_images, train_labels]
    #           val_set: The list [val_images, val_labels] 

def load_images():
    
    sets = ['train', 'val']
    
    data_sets = []
    for dset in sets:
        img_path = './bin_dataset/' + dset + '/ele'
        ele_list = [imread(os.path.join(img_path, img)) for img in os.listdir(img_path)]

        img_path = './bin_dataset/' + dset + '/lio'
        lio_list = [imread(os.path.join(img_path, img)) for img in os.listdir(img_path)]

        set_images = np.stack(ele_list + lio_list)
        N = set_images.shape[0]
        labels = np.ones((N,1))
        labels[0:int(N/2)] = 0
        data_sets.append([set_images, labels])

    train_set, val_set = data_sets

    print("Loaded", len(train_set[0]), "training images")
    print("Loaded", len(val_set[0]), "validation images")
    
    return train_set, val_set


# batchify
    # Inputs:    train_set: List containing images and labels
    #            batch size: The desired size of each batch
    #
    # Returns:   image_batches: A list of shuffled training image batches, each with size batch_size
    #            label_batches: A list of shuffled training label batches, each with size batch_size 

def batchify(train_set, batch_size):
    
    # YOUR CODE HERE
    # initialized two lists
    image_batches = []
    label_batches = []
    
    shuffle_index = np.arange(len(train_set[0]))
    shuffle(shuffle_index)
    
    image_chunk = [None] * batch_size
    label_chunk = [None] * batch_size
    for c in range(0, len(shuffle_index), batch_size):
        for i in range(batch_size):
            image_chunk[i] = train_set[0][shuffle_index[c+i]]
            label_chunk[i] = train_set[1][shuffle_index[c+i]]
        image_batches.append(image_chunk)
        label_batches.append(label_chunk)
    
    return image_batches, label_batches


### Network Functions

In [3]:
def data_transform(data_set):
    data, label = data_set
    data = data.astype(float)
    data /= 256.0
    return [data, label]

#### Activation Functions

In [4]:
# relu
    # Inputs:   x: Multi-dimensional array with size N along the first axis
    # 
    # Returns:  out: Multi-dimensional array with same size of x 

def relu(x):
    
    # YOUR CODE HERE
    out = np.maximum(x, 0)
    return out


# sigmoid
    # Inputs:    x: Multi-dimensional array with size N along the first axis
    # 
    # Returns:   out: Multi-dimensional array with same size of x 

def sigmoid(x):
    
    # YOUR CODE HERE
    out = 1.0/(1 + np.exp(-x))
    return out


# unit_step
    # Inputs:    x: Multi-dimensional array with size N along the first axis 
    # 
    # Returns:   out: Multi-dimensional array with same size of x 

def unit_step(x):
    
    # YOUR CODE HERE
    out = np.heaviside(x, 1)
    return out 

#### Layer Functions

In [5]:
# helper function to calculate 2D convlolution
# input X 2D array with size NxN
# input F 2D array with size fxf
# output out 2D array with size N+f-1 x N+f-1
def convolution2D(X, F):
    N, _ = X.shape
    f, _ = F.shape
    S = N-f+1
    out = np.zeros((S, S))
    for i in range(S):
        for j in range(S):
            out[i, j] = np.sum(F[::-1, ::-1]*X[i:i+f, j:j+f])
    return out
    
# convolve2D
    # Inputs:    X: [N x height x width x num_channels]
    #            filters: [num_filters x filter_height x filter_width x num_input_channels]
    # 
    # Returns:   Xc: output array by convoling X and filters. [N x output_height x output_width x num_filters]

def convolve2D(X0, filters):
   
    N, X0_len, _, num_ch = X0.shape
    num_out_ch, filter_len, _, _ = filters.shape
    F0_side = X0_len - filter_len + 1
    
    F0 = np.zeros((N, F0_side, F0_side, num_out_ch))
    
    for n in range(N):
        for o_ch in range(num_out_ch):
            for ch in range(num_ch):
                # YOUR CODE HERE
                # F0[n, :, :, o_ch] += convolution2D(X0[n, :, :, ch], filters[o_ch, :, :, ch])
                F0[n,0:,0:,o_ch] += signal.convolve2d(X0[n,0:,0:,ch], filters[o_ch, 0:, 0:, ch], mode = "valid")
    return F0


# maxPool
    # Inputs:    R0: [N x height x width x num_channels]
    #            mp_len: size of max pool window, also the stride for this MP
    # 
    # Returns:   p_out: output of pooling R0. [N x output_height x output_width x num_channels]
    #            R0_mask: A binary mask with the same size as R0. Indicates which index was chosen to be the max
    #            for each max pool window. This will be used for backpropagation.

def maxPool(R0, mp_len):

    N, R0_len, _, num_ch = R0.shape
    p_out_len = int((R0_len-mp_len)/mp_len + 1)

    R0_mask = np.zeros(R0.shape)
    p_out = np.zeros((N, p_out_len, p_out_len, num_ch))
    
    for n in range(N):
        for ch in range(num_ch):
            for row in range(p_out_len): 
                for col in range(p_out_len):
                    # YOUR CODE HERE
                    r = row*mp_len
                    c = col*mp_len
                    p_out[n, row, col, ch] = np.amax(R0[n, r:min(r+mp_len, R0_len), c:min(c+mp_len, R0_len), ch])
                    max_idx = np.argmax(R0[n, r:min(r+mp_len, R0_len), c:min(c+mp_len, R0_len), ch])
                    row_idx = max_idx // mp_len + r
                    col_idx = max_idx % mp_len + c
                    R0_mask[n][row_idx][col_idx][ch] = 1
    return p_out, R0_mask

def flatten(Y):
    # flatten function takes a 4D array and output a 2D array
    # input Y: N x N1 x N2 x K
    # output out: N x (N1*N2*K)
    N, N1, N2, K = Y.shape
    out = Y.reshape((N, N1*N2*K))
    return out

# fc
    # Inputs:    X: [N x num_input_features]
    #            W: [num_input_features x num_fc_nodes]
    # 
    # Returns:   out: Linear combination of X and W. [N x num_fc_nodes]

def fc(X, W):
    
    # YOUR CODE HERE
    out = X @ W
    return out

In [6]:
a = np.arange(25).reshape((5, 5))
b = np.arange(9).reshape((3, 3))
C1 = convolution2D(a, b)
C2 = signal.convolve2d(a, b, mode='valid')
print(C1)
print(C2)

[[120. 156. 192.]
 [300. 336. 372.]
 [480. 516. 552.]]
[[120 156 192]
 [300 336 372]
 [480 516 552]]


#### CNN Functions

In [7]:
# cnn_fwd
    # Inputs:    X0: batch of images. [N x img_height x img_width x num_channels]
    #            W0, W1, W2: Parameters of the CNN
    #            mp_len: the length of one side of the max pool window
    # 
    # Returns:   sig: vector containing the output for each sample. [N x 1]
    #            cache: a dict containing the relevant output layer calculations that will be
    #            used in backpropagation
    
def cnn_fwd(X0, W0, W1, W2, mp_len):
    
    # F0 
    # YOUR CODE HERE
    F0 = convolve2D(X0, W0)
    
    # X1p 
    # YOUR CODE HERE
    R0 = relu(F0)
    X1p, R0_mask = maxPool(R0, mp_len)
    
    # X1 (flatten)
    # YOUR CODE HERE
    X1 = flatten(X1p)
    
    # FC Layers
    # YOUR CODE HERE
    F1 = fc(X1, W1)
    
    # Output
    # YOUR CODE HERE
    X2 = relu(F1)
    F2 = fc(X2, W2)
    sig = sigmoid(F2)
    
    # Save outputs of functions for backward pass
    cache = {
        "F0":F0,
        "R0":R0,
        "X1p":X1p,
        "R0m":R0_mask,
        "X1":X1,
        "F1":F1,
        "X2":X2,
        "F2":F2      
    }
    
    return sig, cache


# loss
    # Inputs:    sig: vector containing the CNN output for each sample. [N x 1]
    #            Y: vector containing the ground truth label for each sample. [N x 1]
    # 
    # Returns:   L: Loss/error criterion for the model. 

def loss(sig, Y):
    
    # YOUR CODE HERE
    # cross entropy
    L = 0.0
    N, _ = Y.shape
    for i in range(N):
        L -= Y[i]*np.log(sig[i]) + (1-Y[i])*np.log(1-sig[i])
    L /= N
    return L


### Backprop Functions

In [1]:
# # convolve2DBwd
#     # Inputs:    X0: batch of images. [N x height x width x num_channels]
#     #            dL_dF0: Gradient at the output of the conv layer. 
#     # 
#     # Returns:   dL_dW0. gradient of loss L wrt W0. Same size as W0

# def convolve2DBwd(X0, dL_dF0):
    
#     N, X0_len, _, num_ch = X0.shape
#     _, dL_dF0_len, _, num_out_ch  = dL_dF0.shape
#     filter_len = X0_len - dL_dF0_len + 1
    
#     dL_dW0 = np.zeros((num_out_ch, filter_len, filter_len, num_ch))
    
#     for n in range(N):
#         for o_ch in range(num_out_ch):
#             for ch in range(num_ch):
#                 # YOUR CODE HERE 
    
#     return dL_dW0


# # maxPoolBwd
#     # Inputs:    dL_dX1p: Gradient at the output of the MaxPool layer
#     #            R0_mask: A binary mask with the same size as R0. Defined in maxPool
#     #            mp_len: the length of one side of the max pool window
#     # 
#     # Returns:   dL_dR0: Gradient at the output of ReLu
    
# def maxPoolBwd(dL_dX1p, R0_mask,  mp_len):
    
#     N, H, W, C = R0_mask.shape
#     N, dH, dW, C = dL_dX1p.shape
    
#     dL_dR0 = np.zeros(R0_mask.shape)
    
#     for n in range(N):
#         for ch in range(C):
#             for row in range(dH):
#                 for col in range(dW):
#                     # YOUR CODE HERE
                    
#     return dL_dR0


# # dL_dW2
#     # Inputs:    Y: vector containing the ground truth label for each sample. [N x 1]
#     #            cache: a dict containing the relevant output layer calculations 
#     # 
#     # Returns:   dL_dW2: Gradient of the Loss wrt W2
    
# def dL_dW2(Y, cache):
   
#     # YOUR CODE HERE
    
#     return dL_dW2


# # dL_dW1
#     # Inputs:    Y: vector containing the ground truth label for each sample. [N x 1]
#     #            W2: Weight matrix for the second FC layer
#     #            cache: a dict containing the relevant output layer calculations 
#     # 
#     # Returns:   dL_dW1: Gradient of the Loss wrt W1
    
# def dL_dW1(Y, W2, cache):
    
#     # YOUR CODE HERE
    
#     return dL_dW1


# # dL_dW0
#     # Inputs:    X0: batch of images. [N x height x width x num_channels]
#     #            Y: vector containing the ground truth label for each sample. [N x 1]
#     #            W1: Weight matrix for the first FC layer
#     #            W2: Weight matrix for the second FC layer
#     #            mp_len: the length of one side of the max pool window
#     #            cache: a dict containing the relevant output layer calculations 
#     # 
#     # Returns:   dL_dW0: Gradient of the Loss wrt W0

# def dL_dW0(X0, Y, W1, W2, mp_len, cache):
    
#     N, X1p_len, _, no_out_ch  = cache['X1p'].shape
#     F2 = cache['F2']
#     F1 = cache['F1']
#     R0m = cache['R0m']
#     F0 = cache['F0']
    
#     #dL_dF2
#     # YOUR CODE HERE
    
#     #dL_dF1
#     # YOUR CODE HERE
    
#     #dL_dX1
#     # YOUR CODE HERE
    
#     # dL_dX1p (unflatten)
#     # YOUR CODE HERE
    
#     # dL_dR0 (unpool)
#     # YOUR CODE HERE
    
#     # dL_dF0 (relu_bwd)
#     # YOUR CODE HERE
    
#     # dL_dW0
#     # YOUR CODE HERE
    
#     return dL_dW0
    
        
        
####################################################################################################################

# convolve2DBwd
    # Inputs:    X0: batch of images. [N x height x width x num_channels]
    #            dL_dF0: Gradient at the output of the conv layer. 
    # 
    # Returns:   dL_dW0. gradient of loss L wrt W0. Same size as W0

def convolve2DBwd(X0, dL_dF0):
    
    N, X0_len, _, num_ch = X0.shape
    _, dL_dF0_len, _, num_out_ch  = dL_dF0.shape
    filter_len = X0_len - dL_dF0_len + 1
    
    dL_dW0 = np.zeros((num_out_ch, filter_len, filter_len, num_ch))
    
    for n in range(N):
        for o_ch in range(num_out_ch):
            for ch in range(num_ch):
                # YOUR CODE HERE 
                dL_dW0 = 1 # edit code here
    return dL_dW0

# dL_dF2
    # Inputs:    Y: vector containing the ground truth label for each sample. [N x 1]
    #            cache: a dict containing the relevant output layer calculations 
    # 
    # Returns:   dL_dF2: Gradient of the Loss wrt F2
def dL_dF2(Y, cache):
    
    sig = sigmoid(cache["F2"])
    N =  sig.shape[0]
    dL_dF2 = np.subtract(sig - Y)/N
    print("dL_dF2 should be 800x1, and get ", dL_dF2.shape)
    
    return dL_dF2

# dL_dW2
    # Inputs:    dL_dF2: Gradient of the Loss wrt F2
    #            cache: a dict containing the relevant output layer calculations 
    #            W2: Weight matrix for the second FC layer
    # 
    # Returns:   dL_dW2: Gradient of the Loss wrt W2
    
def dL_dW2(dL_dF2, cache, W2):
   
    # YOUR CODE HERE
    X2 = cache["X2"]
    dL_dW2 = np.matmul(X2.T, dL_dF2)
    dL_dX2 = np.matmul(W2.T, dL_dF2)
    print("dL_dW2 should be 2x1, and get ", dL_dW2.shape)
    print("dL_dX2 should be 800x1, and get ", dL_dX2.shape)
    
    return dL_dW2, dL_dX2

# dL_dF1
    # Inputs:    dL_dX2: Gradient of the Loss wrt X2
    #            cache: a dict containing the relevant output layer calculations 
    # 
    # Returns:   dL_dF1: Gradient of the Loss wrt F1

def dL_dF1(dL_dX2, cache):   
    
    F1 = cache["F1"]
    F1 = np.divide(F1, F1)
    dL_dF1 = np.multiply(dL_dX2, F1)
    print("dL_dF1 should be 800x1, and get ", dL_dF1.shape)
    
    return dL_dF1

# dL_dW1
    # Inputs:    dL_dF1: Gradient of the Loss wrt F1
    #            W1: Weight matrix for the first FC layer
    #            cache: a dict containing the relevant output layer calculations 
    # 
    # Returns:   dL_dW1: Gradient of the Loss wrt W1
    
def dL_dW1(dL_dF1, cache, W1):
    
    # YOUR CODE HERE
    X1 = cache["X1"]
    dL_dW1 = np.matmul(X1.T, dL_dF1)
    dL_dX1 = np.matmul(W1.T, dL_dF1)
    print("dL_dW1 should be 192x2, and get ", dL_dW1.shape)
    print("dL_dX1 should be 800x192, and get ", dL_dX1.shape)
    
    return dL_dW1, dL_dX1

# dL_dX1p
    # Inputs:    dL_dX1: Gradient of the Loss wrt X1
    #
    # Returns:   dL_dX1p: Gradient of the Loss wrt X1p
    
def dL_dX1p(dL_dX1):
    
    dL_dX1p = dL_dX1.reshape((dL_dX1.shape[0], 8, 8, 3)) 
    print("dL_dX1p should be 800x8x8x3, and get ", dL_dX1p.shape)
    
    return dL_dX1p

# dL_dR0
    # Inputs:    dL_dX1p: Gradient of the Loss wrt X1p
    #            cache: a dict containing the relevant output layer calculations 
    #
    # Returns:   dL_dR0: Gradient of the Loss wrt R0
    
def dL_dR0(dL_dX1p, cache):

    R0m = cache['R0m']
    dL_dR0 = R0m
    for n in range(R0m.shape[0]):
        for f in range(R0m.shape[-1]):
            for r in range(R0m.shape[1]):
                for c in range(R0m.shape[2]):
                    dL_dR0[n][r][c][f] = R0m[n][r][c][f] * dL_dX1p[n][r//12][c//12][f]
    print("dL_dR0 should be 800x96x96x3, and get ", dL_dR0.shape)
    
    return dL_dR0

# dL_dF0
    # Inputs:    dL_dR0: Gradient of the Loss wrt R0
    #            cache: a dict containing the relevant output layer calculations 
    # 
    # Returns:   dL_dF0: Gradient of the Loss wrt F0

def dL_dF0(dL_dR0, cache):   
    
    F0 = cache["F0"]
    F0 = np.divide(F0, F0)
    dL_dF0 = np.multiply(dL_dR0, F0)
    print("dL_dF0 should be 800x96x96x3, and get ", dL_dF0.shape)
    
    return dL_dF0
    
# dL_dW0
    # Inputs:    X0: batch of images. [N x height x width x num_channels]
    #            dL_dF0: Gradient of the Loss wrt F0
    #            W0: Weight matrix for the convolutional layer
    # 
    # Returns:   dL_dW0: Gradient of the Loss wrt W0

def dL_dW0(X0, dL_dF0, W0):
    
    dL_dW0 = 1 # edit code here
    
    return dL_dW0
    
        

### Training

#### Load Images

In [8]:
# Load images and scale them
# YOUR CODE HERE
train_set, val_set = load_images()
train_set = data_transform(train_set)
val_set = data_transform(val_set)

Loaded 2000 training images
Loaded 800 validation images


#### Config

In [9]:
# Hyperparameters
epochs = 20
lr = 0.1
batch_size = 16
filter_len = 5
num_out_ch = 3
mp_len = 12
fc_nodes = 2

# Declare weights
# YOUR CODE HERE


In [93]:
for i in range(epochs):
    
    # make set of batches
    # YOUR CODE HERE
    
    for b_idx in range(num_batches):
        X = img_batches[b_idx]
        Y = label_batches[b_idx]
        
        # Forward pass
        # YOUR CODE HERE
        
        # Calculate gradients
        # YOUR CODE HERE
        
        # Update gradients
        # YOUR CODE HERE
      

NameError: name 'num_batches' is not defined

### Test Correctness of Forward and Backward Pass

#### Forward

In [10]:
weights = np.load('weights.npz')
W0 = weights['W0']
W1 = weights['W1']
W2 = weights['W2']

# record the time for the execution
start_time = time.time()
sig, _ = cnn_fwd(val_set[0], W0, W1, W2, mp_len)
train_acc = len(np.where(np.round(sig) == val_set[1])[0])/len(val_set[1])

print("train_loss:", loss(sig, val_set[1]), "train_acc:", train_acc)
print("---total cost is %s seconds ---" % (time.time() - start_time))

train_loss: [0.23266417] train_acc: 0.9175
---total cost is 13.008033037185669 seconds ---


#### Backward

In [ ]:
# Make backprop testing batch
X_bp = np.vstack([train_set[0][0:8,:,:,:], train_set[0][-9:-1,:,:,:]])
Y_bp = np.vstack([train_set[1][0:8], train_set[1][-9:-1]])

# Initialize weights to all ones
# YOUR CODE HERE

# Update weights once
# YOUR CODE HERE


print("W2 value:", np.sum(W2))
print("W1 value:", np.sum(W1))
print("W0 value:", np.sum(W0))